# Segmenting and Clustering Neighborhoods in Toronto
- - -
A peer-graded assignment on Coursera made by **Bach Khoi Vo**.

## Assignment's description

In this assignment, you will be required to explore, segment, and cluster the neighborhoods in the city of Toronto.

For the Toronto neighborhood data, a Wikipedia page exists that has all the information we need to explore and cluster the neighborhoods in Toronto. You will be required to scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas dataframe so that it is in a structured format like the New York dataset.

Once the data is in a structured format, you can replicate the analysis that we did to the New York City dataset to explore and cluster the neighborhoods in the city of Toronto.

In [54]:

import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from pandas.io.json import json_normalize  # tranform JSON file into a pandas dataframe

import folium # map rendering library

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

- - -
## Getting data from Wikipedia

### Raw data

I will get raw HTML source code from the Wikipedia, then store it to `source`. After that, by using BeautifulSoup, I extract contents such as: title, table, etc from the `source`.

In [5]:
url = 'https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&diff=1012118802&oldid=1012023397'
source = requests.get(url=url).text
soup = BeautifulSoup(source, 'lxml')

### Proccessing the data

I will try to find out all of the tables are in the raw data.

In [6]:
tables = soup.find('table', class_ = "wikitable sortable")
# print(tables.prettify())

Converting to the DataFrame

In [45]:
rows = []
columns = ['PostalCode', 'Borough', 'Neighborhood']
for row in tables.find_all('tr'):
    cols = row.find_all('td')
    rows.append([data.text for data in cols])
raw_df = pd.DataFrame(data=rows, columns=columns)
raw_df.head()

,PostalCode,Borough,Neighborhood
0,None,None,None
1,M1A\n,Not assigned\n,Not assigned\n
2,M2A\n,Not assigned\n,Not assigned\n
3,M3A\n,North York\n,Parkwoods\n
4,M4A\n,North York\n,Victoria Village\n


Cleaning the DataFrame

In [46]:
# Drop None & NoneAssigneed Cell
raw_df.dropna(axis=0, inplace=True)
raw_df = raw_df[raw_df['Borough']!='Not assigned\n']
raw_df = raw_df[raw_df['Neighborhood']!='Not assigned\n']
raw_df.head()

,PostalCode,Borough,Neighborhood
3,M3A\n,North York\n,Parkwoods\n
4,M4A\n,North York\n,Victoria Village\n
5,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"
6,M6A\n,North York\n,"Lawrence Manor, Lawrence Heights\n"
7,M7A\n,Downtown Toronto\n,"Queen's Park, Ontario Provincial Government\n"


In [47]:
# Removing '\n'
raw_df = raw_df.apply(lambda row: [col[:-1] for col in row], axis=0)
raw_df.reset_index(inplace=True, drop=True)
raw_df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Finally, we got the cleaned data

In [48]:
df = raw_df.copy()
df.describe()

,PostalCode,Borough,Neighborhood
count,103,103,103
unique,103,11,99
top,M3N,North York,Downsview
freq,1,24,4


- - -
## Get the latitude and the longitude coordinates of each neighborhood
Load the given `Geospatial Coordinates` dataset and join it with the origin.

In [51]:
df_geo = pd.read_csv('Geospatial_Coordinates.csv')
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [52]:
# Join two dataframe
df_toronto = pd.merge(df, df_geo, how='left', left_on='PostalCode', right_on='Postal Code')
df_toronto.head()

,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M3A,North York,Parkwoods,M3A,43.753259,-79.329656
1,M4A,North York,Victoria Village,M4A,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",M5A,43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",M6A,43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",M7A,43.662301,-79.389494


In [53]:
df_toronto.drop('Postal Code', axis=1, inplace=True)
df_toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


- - -
##  Explore and cluster the neighborhoods in Toronto
Before diving into the data, let's quickly examine dataframe

In [56]:
print(f'The dataframe has {len(df_toronto["Borough"].unique())} \
        boroughs and {df_toronto.shape[0]} neighborhoods.')

The dataframe has 11         boroughs and 103 neighborhoods.


### Use geopy library to get the latitude and longitude values of Toronto City.¶

In [58]:
address = "Toronto, ON"

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto city are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto city are 43.6534817, -79.3839347.


### Create a map of the whole Toronto City with neighborhoods superimposed on top.

In [62]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighborhood']):
    label = f'{neighborhood}, {borough}'
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Map of a part of Toronto City
Let's simplify the above map and segment and cluster only the neighborhoods in **North York**. So let's slice the original dataframe and create a new dataframe of the **North York** data.

In [64]:
northyork_data = df_toronto[df_toronto['Borough'] == 'North York'].reset_index(drop=True)
northyork_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
3,M3B,North York,Don Mills,43.745906,-79.352188
4,M6B,North York,Glencairn,43.709577,-79.445073


Let's get the geographical coordinates of **North York**.

In [65]:
address = 'North York, Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.7543263, -79.44911696639593.


Plot again the map and the markers for this region.

In [68]:
# create map of North York using latitude and longitude values
map_northyork = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(northyork_data['Latitude'], northyork_data['Longitude'], northyork_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_northyork)  
    
map_northyork

### Define Foursquare Credentials and Version
For privacy, I will leave this field empty :D

In [69]:
CLIENT_ID = 'JYTGVLK2CM05CTQM11S3YMEYI1I4JYGPUBLOAW4U4S5KENJM' # your Foursquare ID
CLIENT_SECRET = '22JB0FCES5A0AOINYMOUEPFXV53I3TWFZQMABT1ENRZL2J2W' # your Foursquare Secret
VERSION = '20210615' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: JYTGVLK2CM05CTQM11S3YMEYI1I4JYGPUBLOAW4U4S5KENJM
CLIENT_SECRET:22JB0FCES5A0AOINYMOUEPFXV53I3TWFZQMABT1ENRZL2J2W


### Let's explore the first neighborhood in our dataframe.

In [73]:
neighborhood_latitude = northyork_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = northyork_data.loc[0, 'Longitude'] # neighborhood longitude value
neighborhood_name = northyork_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Parkwoods are 43.7532586, -79.3296565.


### Now, let's get the top 100 venues that are in Parkwoods within a radius of 500 meters.

In [74]:
radius = 500
LIMIT = 100
url = f'https://api.foursquare.com/v2/venues/explore?&client_id={CLIENT_ID}\
        &client_secret={CLIENT_SECRET}&v={VERSION}&ll={neighborhood_latitude},{neighborhood_longitude}\
        &radius={radius}&limit={LIMIT}'
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60c9b9eab344126cb4c70fb2'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Parkwoods - Donalda',
  'headerFullLocation': 'Parkwoods - Donalda, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 3,
  'suggestedBounds': {'ne': {'lat': 43.757758604500005,
    'lng': -79.32343823984928},
   'sw': {'lat': 43.7487585955, 'lng': -79.33587476015072}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e6696b6d16433b9ffff47c3',
       'name': 'KFC',
       'location': {'lat': 43.75438666345904,
        'lng': -79.3330206627504,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.75

From the Foursquare lab in the previous module, we know that all the information is in the _items_ key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.


In [75]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a _pandas_ dataframe.


In [76]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

<ipython-input-76-561c05f0fdd1>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,KFC,Fast Food Restaurant,43.754387,-79.333021
1,Brookbanks Park,Park,43.751976,-79.332140
2,Variety Store,Food & Drink Shop,43.751974,-79.333114


- - - 
## Explore Neighborhoods in North York 

### Let's create a function to repeat the same process to all the neighborhoods in North York

In [78]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Now write the code to run the above function on each neighborhood and create a new dataframe called _northyork_venues_.

In [79]:
northyork_venues = getNearbyVenues(names=northyork_data['Neighborhood'],
                                   latitudes=northyork_data['Latitude'],
                                   longitudes=northyork_data['Longitude'])

Parkwoods
Victoria Village
Lawrence Manor, Lawrence Heights
Don Mills
Glencairn
Don Mills
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Fairview, Henry Farm, Oriole
Northwood Park, York University
Bayview Village
Downsview
York Mills, Silver Hills
Downsview
North Park, Maple Leaf Park, Upwood Park
Humber Summit
Willowdale, Newtonbrook
Downsview
Bedford Park, Lawrence Manor East
Humberlea, Emery
Willowdale, Willowdale East
Downsview
York Mills West
Willowdale, Willowdale West


Let's check the resulting dataframe

In [80]:
northyork_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
1,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


Let's check how many venues were returned for each neighborhood.

In [81]:
northyork_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bathurst Manor, Wilson Heights, Downsview North",17,17,17,17,17,17
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",23,23,23,23,23,23
Don Mills,20,20,20,20,20,20
Downsview,18,18,18,18,18,18
"Fairview, Henry Farm, Oriole",64,64,64,64,64,64
Glencairn,4,4,4,4,4,4
Hillcrest Village,6,6,6,6,6,6
Humber Summit,4,4,4,4,4,4


Let's find out how many unique categories can be curated from all the returned venues

In [89]:
print(f'There are {northyork_venues["Venue Category"].unique().size} unique categories!')

There are 89 unique categories!


- - -
## Analyze Each Neighborhood

In [91]:
# one hot encoding
northyork_onehot = pd.get_dummies(northyork_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
northyork_onehot['Neighborhood'] = northyork_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [northyork_onehot.columns[-1]] + list(northyork_onehot.columns[:-1])
northyork_onehot = northyork_onehot[fixed_columns]

northyork_onehot.head()

,Neighborhood,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,...,Sporting Goods Shop,Supermarket,Supplement Shop,Sushi Restaurant,Thai Restaurant,Toy / Game Store,Trail,Video Game Store,Vietnamese Restaurant,Women's Store
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [92]:
northyork_onehot.shape

(227, 90)

### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [94]:
northyork_grouped = northyork_onehot.groupby('Neighborhood').mean().reset_index()
northyork_grouped.head()

,Neighborhood,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,...,Sporting Goods Shop,Supermarket,Supplement Shop,Sushi Restaurant,Thai Restaurant,Toy / Game Store,Trail,Video Game Store,Vietnamese Restaurant,Women's Store
0,"Bathurst Manor, Wilson Heights, Downsview North",0.000000,0.000000,0.00,0.0,0.00,0.000000,0.0,0.117647,0.0,...,0.00,0.00,0.0,0.058824,0.000000,0.0,0.0,0.0,0.0,0.0
1,Bayview Village,0.000000,0.000000,0.00,0.0,0.00,0.000000,0.0,0.250000,0.0,...,0.00,0.00,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
2,"Bedford Park, Lawrence Manor East",0.000000,0.043478,0.00,0.0,0.00,0.000000,0.0,0.000000,0.0,...,0.00,0.00,0.0,0.043478,0.043478,0.0,0.0,0.0,0.0,0.0
3,Don Mills,0.000000,0.000000,0.05,0.0,0.05,0.000000,0.0,0.000000,0.0,...,0.05,0.05,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
4,Downsview,0.055556,0.000000,0.00,0.0,0.00,0.055556,0.0,0.055556,0.0,...,0.00,0.00,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0


In [95]:
northyork_grouped.shape

(19, 90)

### Let's print each neighborhood along with the top 5 most common venues

In [96]:
num_top_venues = 5

for hood in northyork_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = northyork_grouped[northyork_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bathurst Manor, Wilson Heights, Downsview North----
            venue  freq
0            Bank  0.12
1     Coffee Shop  0.12
2     Gas Station  0.06
3  Ice Cream Shop  0.06
4            Park  0.06


----Bayview Village----
                 venue  freq
0   Chinese Restaurant  0.25
1                 Bank  0.25
2                 Café  0.25
3  Japanese Restaurant  0.25
4              Airport  0.00


----Bedford Park, Lawrence Manor East----
            venue  freq
0     Coffee Shop  0.09
1  Sandwich Place  0.09
2      Restaurant  0.09
3   Grocery Store  0.04
4            Café  0.04


----Don Mills----
            venue  freq
0             Gym  0.15
1      Restaurant  0.10
2      Beer Store  0.05
3     Coffee Shop  0.05
4  Clothing Store  0.05


----Downsview----
                        venue  freq
0               Grocery Store  0.17
1               Shopping Mall  0.11
2                        Park  0.11
3  Construction & Landscaping  0.06
4                Liquor Store  0.06


----Fairvi

###  Let's put that into a pandas dataframe

In [97]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [114]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = northyork_grouped['Neighborhood']

for ind in np.arange(northyork_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(northyork_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Gas Station,Ice Cream Shop,Park,Intersection,Pharmacy,Gift Shop,Sushi Restaurant,Pizza Place
1,Bayview Village,Chinese Restaurant,Bank,Café,Japanese Restaurant,Airport,Liquor Store,Movie Theater,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant
2,"Bedford Park, Lawrence Manor East",Coffee Shop,Sandwich Place,Restaurant,Grocery Store,Café,Indian Restaurant,American Restaurant,Greek Restaurant,Italian Restaurant,Liquor Store
3,Don Mills,Gym,Restaurant,Beer Store,Coffee Shop,Clothing Store,Chinese Restaurant,Caribbean Restaurant,Café,Sporting Goods Shop,Supermarket
4,Downsview,Grocery Store,Shopping Mall,Park,Construction & Landscaping,Liquor Store,Hotel,Gym / Fitness Center,Food Truck,Electronics Store,Discount Store


## Cluster Neighborhoods

Run k-means to cluster the neighborhood into 5 clusters.

In [115]:
# set number of clusters
kclusters = 5

northyork_grouped_clustering = northyork_grouped.drop('Neighborhood', axis=1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(northyork_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 4, 1, 4, 3], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [116]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

northyork_merged = northyork_data

northyork_merged = northyork_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

northyork_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,0.0,Food & Drink Shop,Fast Food Restaurant,Park,Airport,Korean Restaurant,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Metro Station,Mediterranean Restaurant
1,M4A,North York,Victoria Village,43.725882,-79.315572,1.0,Pizza Place,Hockey Arena,Coffee Shop,Portuguese Restaurant,Korean Restaurant,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Metro Station,Mediterranean Restaurant
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,1.0,Clothing Store,Women's Store,Vietnamese Restaurant,Athletics & Sports,Miscellaneous Shop,Coffee Shop,Furniture / Home Store,Boutique,Liquor Store,Movie Theater
3,M3B,North York,Don Mills,43.745906,-79.352188,1.0,Gym,Restaurant,Beer Store,Coffee Shop,Clothing Store,Chinese Restaurant,Caribbean Restaurant,Café,Sporting Goods Shop,Supermarket
4,M6B,North York,Glencairn,43.709577,-79.445073,4.0,Pizza Place,Park,Metro Station,Movie Theater,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Mediterranean Restaurant,Luggage Store,Liquor Store


In [121]:
northyork_merged.dropna(axis=0, inplace=True)

In [123]:
northyork_merged['Cluster Labels'] = northyork_merged['Cluster Labels'].astype('int64')

Finally, let's visualize the resulting clusters

In [124]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(northyork_merged['Latitude'], northyork_merged['Longitude'], northyork_merged['Neighborhood'], northyork_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters

#### Cluster 1

In [125]:
northyork_merged.loc[northyork_merged['Cluster Labels'] == 0, northyork_merged.columns[[1] + list(range(5, northyork_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,0,Food & Drink Shop,Fast Food Restaurant,Park,Airport,Korean Restaurant,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Metro Station,Mediterranean Restaurant
16,North York,0,Park,Home Service,Korean Restaurant,Movie Theater,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Metro Station,Mediterranean Restaurant,Luggage Store


#### Cluster 2

In [126]:
northyork_merged.loc[northyork_merged['Cluster Labels'] == 1, northyork_merged.columns[[1] + list(range(5, northyork_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,1,Pizza Place,Hockey Arena,Coffee Shop,Portuguese Restaurant,Korean Restaurant,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Metro Station,Mediterranean Restaurant
2,North York,1,Clothing Store,Women's Store,Vietnamese Restaurant,Athletics & Sports,Miscellaneous Shop,Coffee Shop,Furniture / Home Store,Boutique,Liquor Store,Movie Theater
3,North York,1,Gym,Restaurant,Beer Store,Coffee Shop,Clothing Store,Chinese Restaurant,Caribbean Restaurant,Café,Sporting Goods Shop,Supermarket
5,North York,1,Gym,Restaurant,Beer Store,Coffee Shop,Clothing Store,Chinese Restaurant,Caribbean Restaurant,Café,Sporting Goods Shop,Supermarket
6,North York,1,Fast Food Restaurant,Golf Course,Athletics & Sports,Pool,Mediterranean Restaurant,Dog Run,Korean Restaurant,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant
7,North York,1,Bank,Coffee Shop,Gas Station,Ice Cream Shop,Park,Intersection,Pharmacy,Gift Shop,Sushi Restaurant,Pizza Place
8,North York,1,Clothing Store,Coffee Shop,Fast Food Restaurant,Restaurant,Cosmetics Shop,Shoe Store,Women's Store,Toy / Game Store,Food Court,Japanese Restaurant
9,North York,1,Vietnamese Restaurant,Coffee Shop,Miscellaneous Shop,Bar,Furniture / Home Store,Caribbean Restaurant,Metro Station,Airport,Liquor Store,Movie Theater
10,North York,1,Chinese Restaurant,Bank,Café,Japanese Restaurant,Airport,Liquor Store,Movie Theater,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant
11,North York,1,Grocery Store,Shopping Mall,Park,Construction & Landscaping,Liquor Store,Hotel,Gym / Fitness Center,Food Truck,Electronics Store,Discount Store


#### Cluster 3

In [127]:
northyork_merged.loc[northyork_merged['Cluster Labels'] == 2, northyork_merged.columns[[1] + list(range(5, northyork_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,North York,2,Convenience Store,Construction & Landscaping,Park,Movie Theater,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Metro Station,Mediterranean Restaurant,Luggage Store


#### Cluster 4

In [128]:
northyork_merged.loc[northyork_merged['Cluster Labels'] == 3, northyork_merged.columns[[1] + list(range(5, northyork_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,North York,3,Baseball Field,Airport,Juice Bar,Movie Theater,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Metro Station,Mediterranean Restaurant,Luggage Store


#### Cluster 5

In [129]:
northyork_merged.loc[northyork_merged['Cluster Labels'] == 4, northyork_merged.columns[[1] + list(range(5, northyork_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,North York,4,Pizza Place,Park,Metro Station,Movie Theater,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Mediterranean Restaurant,Luggage Store,Liquor Store
15,North York,4,Shopping Mall,Home Service,Pizza Place,Gym,Hotel,Liquor Store,Hockey Arena,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant
